# Article -> Country (Coordinate-based)
This notebook computes what countries are associated with each Wikipedia article via geolocation of the latitude-longitude coordinates extracted from Wikidata. It builds on a notebook that computes countries based on various cultural properties on Wikidata.

## Setup

In [1]:
# using custom conda environment so I can include shapely for the coordinate -> country look-up
!which python

/home/isaacj/.conda/envs/2024-04-29T19.33.29_isaacj/bin/python


In [ ]:
!conda install shapely --channel conda-forge

In [2]:
import csv
import json
import os
import sys

import requests
from shapely.geometry import shape, Point

import wmfdata

In [2]:
spark = wmfdata.spark.create_session(app_name='pyspark large; regions; isaacj',
                                  type='yarn-large', # local, yarn-regular, yarn-large
                                  ship_python_env=True,  # this will include shapely library
                                  extra_settings={
                                      "spark.executor.memory": "12g",
                                      "spark.executor.cores": 2}
                                  )  

The requested conda environment has already been packed.
If you want it to be repacked, set force=True in conda_pack_kwargs.
Shipping conda-2024-04-29T19.33.29_isaacj.tgz to remote Spark executors.
SPARK_HOME: /usr/lib/spark3
Using Hadoop client lib jars at 3.2.0, provided by Spark.
PYSPARK_PYTHON=conda-2024-04-29T19.33.29_isaacj/bin/python3


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/29 20:57:57 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
24/04/29 20:57:58 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
24/04/29 20:57:58 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
24/04/29 20:57:58 WARN Utils: Service 'sparkDriver' could not bind on port 12002. Attempting port 12003.
24/04/29 20:57:58 WARN Utils: Service 'sparkDriver' could not bind on port 12003. Attempting port 12004.
24/04/29 20:57:58 WARN Utils: Service 'sparkDriver' could not bind on port 12004. Attempting port 12005.
24/04/29 20:57:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/29 20:57:58 WARN Utils: Service 'SparkUI' c

## Load in supporting data files from Github

In [4]:
QID_TO_REGION = {}
QID_TO_GEOMETRY = {}
COUNTRY_PROPERTIES = {
    "P17": "country",
    "P19": "place of birth",
    "P27": "country of citizenship",
    "P131": "located in the administrative territorial entity",
    "P183": "endemic to",
    "P361": "part of",
    "P495": "country of origin",
    "P1269": "facet of",
    "P1532": "country for sport",
    "P3842": "located in present-day administrative territorial entity",
    }

In [4]:
data_dir = './data'
aggregation_tsv = os.path.join(data_dir, 'country_aggregation.tsv')
aggregation_url = "https://github.com/geohci/wiki-region-groundtruth/raw/main/resources/country_aggregation.tsv"
aggregation_header = ['Aggregation', 'From', 'QID To', 'QID From']
agg_qid_to_idx = aggregation_header.index("QID To")
agg_qid_from_idx = aggregation_header.index("QID From")

countries_tsv = os.path.join(data_dir, "countries.tsv")
countries_url = "https://raw.githubusercontent.com/wikimedia-research/canonical-data/master/country/countries.tsv"
countries_header = ['name', 'iso_code', 'iso_alpha3_code', 'wikidata_id', 'is_protected', 'data_risk_score', 'data_risk_classification',
                    'maxmind_continent', 'un_continent', 'un_subcontinent', 'un_m49_code', 'wikimedia_region', 'grant_committee_region',
                    'form_990_region', 'economic_region', 'emerging_classification', 'is_eu', 'is_un_member', 'is_un_data_entity',
                    'is_imf_data_entity', 'is_world_bank_data_entity', 'is_penn_world_table_data_entity', 'market_research_classification']
country_qid_idx = countries_header.index("wikidata_id")
country_name_idx = countries_header.index("name")

region_geoms_geojson = os.path.join(data_dir, "ne_10m_admin_0_map_units.geojson")
region_geoms_url = "https://github.com/geohci/wiki-region-groundtruth/raw/main/resources/ne_10m_admin_0_map_units.geojson"

In [5]:
if not os.path.exists(countries_tsv):
    response = requests.get(countries_url)
    with open(countries_tsv, mode="wb") as fout:
        fout.write(response.content)
        
if not os.path.exists(aggregation_tsv):
    response = requests.get(aggregation_url)
    with open(aggregation_tsv, mode="wb") as fout:
        fout.write(response.content)
        
# load in geometries for the regions identified via Wikidata
if not os.path.exists(region_geoms_geojson):
    response = requests.get(region_geoms_url)
    with open(region_geoms_geojson, mode="wb") as fout:
        fout.write(response.content)

In [6]:
# load in canonical mapping of QID -> region name for labeling
with open(countries_tsv, 'r') as fin:
    assert next(fin).strip().split('\t') == countries_header
    for line in fin:
        row = line.strip().split("\t")
        qid = row[country_qid_idx]
        region_name = row[country_name_idx]
        QID_TO_REGION[qid] = region_name
print(f"Loaded {len(QID_TO_REGION)} QID-region pairs for matching against Wikidata -- e.g., Q31: {QID_TO_REGION['Q31']}")
    
with open(aggregation_tsv, 'r') as fin:
    assert next(fin).strip().split("\t") == aggregation_header
    for line in fin:
        row = line.strip().split("\t")
        qid_to = row[agg_qid_to_idx]
        qid_from = row[agg_qid_from_idx]
        if qid_to in QID_TO_REGION:
            # map new QID to valid country
            # e.g., QID for West Bank -> Palestine
            QID_TO_REGION[qid_from] = QID_TO_REGION[qid_to]
print(f"Now {len(QID_TO_REGION)} QID-region pairs after adding aggregations -- e.g., Q40362: {QID_TO_REGION['Q40362']}")

with open(region_geoms_geojson, 'r') as fin:
    regions = json.load(fin)['features']
    for c in regions:
        qid = c['properties']['WIKIDATAID']
        if qid in QID_TO_REGION:
            QID_TO_GEOMETRY[qid] = shape(c['geometry'])
        else:
            print(f"Skipping geometry for: {c['properties']['NAME']} ({qid})")

for qid in QID_TO_REGION:
    if qid not in QID_TO_GEOMETRY:
        alt_found = False
        country = QID_TO_REGION[qid]
        for alt_qid in QID_TO_REGION:
            if QID_TO_REGION[alt_qid] == country:
                if alt_qid in QID_TO_GEOMETRY:
                    alt_found = True
                    break
        if not alt_found:
            print(f"Missing geometry: {QID_TO_REGION[qid]} ({qid})")

Loaded 250 QID-region pairs for matching against Wikidata -- e.g., Q31: Belgium
Now 304 QID-region pairs after adding aggregations -- e.g., Q40362: Western Sahara
Skipping geometry for: Dhekelia (Q9206745)
Skipping geometry for: UNDOF Zone (Q1428532)
Skipping geometry for: Korean DMZ (south) (Q331990)
Skipping geometry for: Korean DMZ (north) (Q331990)
Skipping geometry for: Iraqi Kurdistan (Q205047)
Skipping geometry for: USNB Guantanamo Bay (Q762570)
Skipping geometry for: N. Cyprus (Q23681)
Skipping geometry for: Cyprus U.N. Buffer Zone (Q116970)
Skipping geometry for: Siachen Glacier (Q333946)
Skipping geometry for: Akrotiri (Q9143535)
Skipping geometry for: Paracel Is. (Q274388)
Skipping geometry for: Spratly Is. (Q215664)
Skipping geometry for: Clipperton I. (Q161258)
Skipping geometry for: Bajo Nuevo Bank (Q1257783)
Skipping geometry for: Serranilla Bank (Q1169008)
Skipping geometry for: Scarborough Reef (Q628716)


## Functions

In [7]:
def pointInCountry(lon, lat):
    """Determine which region contains a lat-lon coordinate.
    
    Depends on shapely library and region_shapes object, which contains a dictionary
    mapping QIDs to shapely geometry objects.
    """
    try:
        pt = Point(lon, lat)
        for qid in QID_TO_GEOMETRY:
            if QID_TO_GEOMETRY[qid].contains(pt):
                return QID_TO_REGION[qid]
    except Exception:
        pass
    return None
    
spark.udf.register('pointInCountry', pointInCountry, 'String')

<function __main__.pointInCountry(lon, lat)>

In [8]:
# value info in wikidata entity table (https://wikitech.wikimedia.org/wiki/Analytics/Data_Lake/Edits/Wikidata_entity)
# is a string as opposed to struct (because it has a variable schema)
# this UDF extracts the QID value (or null if doesn't exist)
def getWikidataValue(obj):
    try:
        d = json.loads(obj)
        return d.get('id')
    except Exception:
        return None
    
spark.udf.register('getWikidataValue', getWikidataValue, 'String')

# specific functions for getting lat and lon out of the P625 property in the Wikidata entity
def getLat(obj):
    try:
        d = json.loads(obj)
        return d.get('latitude')
    except Exception:
        return None
    
def getLon(obj):
    try:
        d = json.loads(obj)
        return d.get('longitude')
    except Exception:
        return None
    
spark.udf.register('getLat', getLat, 'Float')
spark.udf.register('getLon', getLon, 'Float')

<function __main__.getLon(obj)>

## Compute data

In [9]:
print("Wikidata snapshots:")
spark.sql('SHOW PARTITIONS wmf.wikidata_item_page_link').show(50, False)

print("\nMediawiki snapshots:")
spark.sql('SHOW PARTITIONS wmf_raw.mediawiki_project_namespace_map').show(50, False)

Wikidata snapshots:
+-------------------+
|partition          |
+-------------------+
|snapshot=2024-02-26|
|snapshot=2024-03-04|
|snapshot=2024-03-11|
|snapshot=2024-03-18|
|snapshot=2024-03-25|
|snapshot=2024-04-01|
|snapshot=2024-04-08|
|snapshot=2024-04-15|
+-------------------+


Mediawiki snapshots:
+------------------------+
|partition               |
+------------------------+
|snapshot=2016-12_private|
|snapshot=2017-07_private|
|snapshot=2023-10        |
|snapshot=2023-11        |
|snapshot=2023-12        |
|snapshot=2024-01        |
|snapshot=2024-02        |
|snapshot=2024-03        |
+------------------------+



In [5]:
mediawiki_snapshot = '2024-03'
wikidata_snapshot = '2024-04-01'
prop_list = tuple(COUNTRY_PROPERTIES)
print(prop_list)
tablename = 'isaacj.qid_to_country'

('P17', 'P19', 'P27', 'P131', 'P183', 'P361', 'P495', 'P1269', 'P1532', 'P3842')


In [11]:
create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {tablename} (
        qid              STRING  COMMENT 'Wikidata ID of item with at least one Wikipedia sitelink -- e.g., Q42',
        property         STRING  COMMENT 'Wikidata property (e.g., P625 for coordinates) from which country was derived',
        country          STRING  COMMENT 'Region name'
    )
    PARTITIONED BY (
        snapshot         STRING  COMMENT 'Wikidata snapshot processed'
    )
    """

print(create_table_query)
spark.sql(create_table_query)


    CREATE TABLE IF NOT EXISTS isaacj.qid_to_country (
        qid              STRING  COMMENT 'Wikidata ID of item with at least one Wikipedia sitelink -- e.g., Q42',
        property         STRING  COMMENT 'Wikidata property (e.g., P625 for coordinates) from which country was derived',
        country          STRING  COMMENT 'Region name'
    )
    PARTITIONED BY (
        snapshot         STRING  COMMENT 'Wikidata snapshot processed'
    )
    


24/04/29 20:58:54 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
24/04/29 20:58:54 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


DataFrame[]

In [12]:
"""
Full query is below that grabs country data from a set of ~10 properties
AND grabs country data based on an item's coordinates

Explanation of CTEs:
* relevant_wikis: get list of Wikipedia wiki_dbs (e.g., enwiki) so as to limit the
    the Wikidata items considered to just those with Wikipedia sitelinks
* relevant_qids: get set of Wikidata item IDs that have at least one Wikipedia sitelink
* exploded_statements: explode Wikidata entity data to one Wikidata claim per row
* lat_lon_coords: extract lat/lon values from claims to be geolocated
* geolocated: pass lat/lon values to UDF to identify which country they are in
* INSERT: add to table
"""

query = f"""
WITH relevant_wikis AS (
    SELECT
      DISTINCT(database_code) AS wiki_db
    FROM canonical_data.wikis
    WHERE
      database_group = 'wikipedia'
      AND status = 'open'
      AND visibility = 'public'
      AND editability = 'public'
),
relevant_qids AS (
    SELECT
      DISTINCT(item_id) AS item_id
    FROM wmf.wikidata_item_page_link wd
    INNER JOIN relevant_wikis db
      ON (wd.wiki_db = db.wiki_db)
    WHERE
      snapshot = '{wikidata_snapshot}'
      AND page_namespace = 0
),
exploded_statements AS (
    SELECT
      id AS item_id,
      explode(claims) AS claim
    FROM wmf.wikidata_entity w
    INNER JOIN relevant_qids q
      ON (w.id = q.item_id)
    WHERE
      w.snapshot = '{wikidata_snapshot}'
),
lat_lon_coords AS (
    SELECT
      item_id,
      getLat(claim.mainSnak.dataValue.value) as lat,
      getLon(claim.mainSnak.dataValue.value) as lon
    FROM exploded_statements
    WHERE
      claim.mainSnak.property = 'P625'
),
geolocated AS (
    SELECT
      item_id,
      pointInCountry(lon, lat) AS country
    FROM lat_lon_coords
)
INSERT INTO TABLE {tablename}
PARTITION(snapshot='{wikidata_snapshot}')
SELECT
  item_id,
  "P625" AS property,
  country
FROM geolocated
WHERE
  country IS NOT NULL
"""

print(query)
result = spark.sql(query)


WITH relevant_wikis AS (
    SELECT
      DISTINCT(database_code) AS wiki_db
    FROM canonical_data.wikis
    WHERE
      database_group = 'wikipedia'
      AND status = 'open'
      AND visibility = 'public'
      AND editability = 'public'
),
relevant_qids AS (
    SELECT
      DISTINCT(item_id) AS item_id
    FROM wmf.wikidata_item_page_link wd
    INNER JOIN relevant_wikis db
      ON (wd.wiki_db = db.wiki_db)
    WHERE
      snapshot = '2024-04-01'
      AND page_namespace = 0
),
exploded_statements AS (
    SELECT
      id AS item_id,
      explode(claims) AS claim
    FROM wmf.wikidata_entity w
    INNER JOIN relevant_qids q
      ON (w.id = q.item_id)
    WHERE
      w.snapshot = '2024-04-01'
),
lat_lon_coords AS (
    SELECT
      item_id,
      getLat(claim.mainSnak.dataValue.value) as lat,
      getLon(claim.mainSnak.dataValue.value) as lon
    FROM exploded_statements
    WHERE
      claim.mainSnak.property = 'P625'
),
geolocated AS (
    SELECT
      item_id,
      point

24/04/29 20:59:12 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/04/29 20:59:27 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/04/29 21:07:27 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 118 for reason Container killed by YARN for exceeding physical memory limits. 13.2 GB of 13.2 GB physical memory used. Consider boosting spark.executor.memoryOverhead.
24/04/29 21:07:27 ERROR YarnScheduler: Lost executor 118 on an-worker1160.eqiad.wmnet: Container killed by YARN for exceeding physical memory limits. 13.2 GB of 13.2 GB physical memory used. Consider boosting spark.executor.memoryOverhead.
24/04/29 21:07:27 WARN TaskSetManager: Lost task 324.0 in stage 4.0 (TID 1542) (an-worker1160.eqiad.wmnet executor 118): ExecutorLostFailure (ex

In [13]:
# check output
# NOTE: this was done after the cultural ones had already been
#       added to the table so the output reflects that too.
spark.sql(f"""
WITH enwiki_qids AS (
    SELECT
      DISTINCT(item_id)
    FROM wmf.wikidata_item_page_link
    WHERE
      snapshot = "{wikidata_snapshot}"
      AND wiki_db = "enwiki"
)
SELECT
  country,
  COUNT(DISTINCT(qid)) AS num_qids,
  COUNT(1) AS num_rows
FROM {tablename} all
INNER JOIN enwiki_qids e
  ON (all.qid = e.item_id)
WHERE
  snapshot = "{wikidata_snapshot}"
GROUP BY
  country
ORDER BY
  num_qids DESC
""").show(500, False)


+---------------------------------------------+--------+--------+
|country                                      |num_qids|num_rows|
+---------------------------------------------+--------+--------+
|United States                                |1140647 |1496715 |
|United Kingdom                               |379518  |507887  |
|France                                       |165527  |227448  |
|India                                        |163140  |223625  |
|Canada                                       |153888  |197485  |
|Australia                                    |129641  |169231  |
|Germany                                      |125971  |171207  |
|Japan                                        |98568   |127913  |
|Poland                                       |86050   |157556  |
|Italy                                        |85579   |113187  |
|Russia                                       |73587   |120024  |
|Iran                                         |70274   |125166  |
|Spain    

In [15]:
# check output
# NOTE: this was done after the cultural ones had already been
#       added to the table so the output reflects that too.
spark.sql(f"""
SELECT
  property,
  COUNT(1) AS num_instances
FROM {tablename}
WHERE
  snapshot = "{wikidata_snapshot}"
GROUP BY
  property
""").show(500, False)

+--------+-------------+
|property|num_instances|
+--------+-------------+
|P183    |20130        |
|P3842   |518          |
|P625    |5931926      |
|P27     |2936497      |
|P1532   |174605       |
|P19     |87022        |
|P495    |741334       |
|P361    |828          |
|P1269   |18970        |
|P17     |8378688      |
|P131    |165811       |
+--------+-------------+



## Dump

In [16]:
groundtruth_query = f"""
SELECT
  qid,
  CONCAT_WS('|', COLLECT_SET(country)) AS countries
FROM {tablename}
WHERE
  snapshot = "{wikidata_snapshot}"
GROUP BY
  qid
"""
print(groundtruth_query)
spark.sql(groundtruth_query).coalesce(1).write.csv(
    path=f"/user/isaacj/region-groundtruth-{wikidata_snapshot}",
    compression="gzip",
    header=True,
    sep="\t")


SELECT
  qid,
  CONCAT_WS('|', COLLECT_SET(country)) AS countries
FROM isaacj.qid_to_country
WHERE
  snapshot = "2024-04-01"
GROUP BY
  qid



In [17]:
!hdfs dfs -ls "region-groundtruth-{wikidata_snapshot}"

Found 2 items
-rw-r-----   3 isaacj isaacj          0 2024-04-29 21:54 region-groundtruth-2024-04-01/_SUCCESS
-rw-r-----   3 isaacj isaacj   61610088 2024-04-29 21:54 region-groundtruth-2024-04-01/part-00000-b246a0e2-e0e1-429f-a3fb-ae2da5711b4b-c000.csv.gz


In [18]:
!hdfs dfs -text region-groundtruth-2024-04-01/part-00000-b246a0e2-e0e1-429f-a3fb-ae2da5711b4b-c000.csv.gz | head


qid	countries
Q1000098	Japan
Q1001499	Slovakia
Q100209	United States|Germany
Q100223	Germany
Q1002412	Germany
Q100252590	Germany
Q100259443	Egypt
Q100271418	Saudi Arabia
Q1003019	United States
text: Unable to write to output stream.


In [ ]:
!hdfs dfs -copyToLocal "region-groundtruth-2024-04-01/part-00000-b246a0e2-e0e1-429f-a3fb-ae2da5711b4b-c000.csv.gz" "./region-groundtruth-2024-04-01.tsv.gz"
!chmod 644 "./region-groundtruth-2024-04-01.tsv.gz"

## Incidence Rate

In [1]:
import wmfdata

In [2]:
spark = wmfdata.spark.create_session(app_name='pyspark reg; regions; isaacj',
                                  type='yarn-regular', # local, yarn-regular, yarn-large
                                  )  

SPARK_HOME: /usr/lib/spark3
Using Hadoop client lib jars at 3.2.0, provided by Spark.
PYSPARK_PYTHON=/opt/conda-analytics/bin/python3


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/30 17:13:55 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
24/04/30 17:13:56 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
24/04/30 17:13:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/30 17:14:04 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 13000. Attempting port 13001.
24/04/30 17:14:04 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


In [3]:
wikidata_snapshot = '2024-04-01'
tablename = 'isaacj.qid_to_country'

In [6]:
"""
Base incidence rate -- equally-weighted articles
"""

query = f"""
WITH relevant_wikis AS (
    SELECT
      DISTINCT(database_code) AS wiki_db
    FROM canonical_data.wikis
    WHERE
      database_group = 'wikipedia'
      AND status = 'open'
      AND visibility = 'public'
      AND editability = 'public'
),
overall AS (
    SELECT
      COUNT(DISTINCT(item_id)) AS num_items
    FROM wmf.wikidata_item_page_link wd
    INNER JOIN relevant_wikis db
      ON (wd.wiki_db = db.wiki_db)
    WHERE
      snapshot = '{wikidata_snapshot}'
      AND page_namespace = 0
),
groundtruth AS (
    SELECT DISTINCT
      qid,
      country
    FROM {tablename}
    WHERE
      snapshot = '{wikidata_snapshot}'
),
overall_country_counts AS (
   SELECT
      country,
      COUNT(1) AS num_items
    FROM groundtruth
    GROUP BY
      country
)
SELECT
  country,
  c.num_items AS num_qids,
  ROUND(c.num_items / o.num_items, 8) AS term_freq
FROM overall_country_counts c
CROSS JOIN overall o
ORDER BY
  term_freq DESC
"""

print(query)
spark.sql(query).show(300, False)


WITH relevant_wikis AS (
    SELECT
      DISTINCT(database_code) AS wiki_db
    FROM canonical_data.wikis
    WHERE
      database_group = 'wikipedia'
      AND status = 'open'
      AND visibility = 'public'
      AND editability = 'public'
),
overall AS (
    SELECT
      COUNT(DISTINCT(item_id)) AS num_items
    FROM wmf.wikidata_item_page_link wd
    INNER JOIN relevant_wikis db
      ON (wd.wiki_db = db.wiki_db)
    WHERE
      snapshot = '2024-04-01'
      AND page_namespace = 0
),
groundtruth AS (
    SELECT DISTINCT
      qid,
      country
    FROM isaacj.qid_to_country
    WHERE
      snapshot = '2024-04-01'
),
overall_country_counts AS (
   SELECT
      country,
      COUNT(1) AS num_items
    FROM groundtruth
    GROUP BY
      country
)
SELECT
  country,
  c.num_items AS num_qids,
  ROUND(c.num_items / o.num_items, 8) AS term_freq
FROM overall_country_counts c
CROSS JOIN overall o
ORDER BY
  term_freq DESC



24/04/30 16:57:04 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


+---------------------------------------------+--------+----------+
|country                                      |num_qids|term_freq |
+---------------------------------------------+--------+----------+
|United States                                |1787968 |0.08084151|
|Germany                                      |629206  |0.02844903|
|Japan                                        |489558  |0.02213496|
|France                                       |486498  |0.02199661|
|Russia                                       |466512  |0.02109296|
|United Kingdom                               |435024  |0.01966925|
|Norway                                       |433766  |0.01961237|
|Canada                                       |396692  |0.0179361 |
|Mexico                                       |372437  |0.01683943|
|Spain                                        |371801  |0.01681068|
|India                                        |357697  |0.01617298|
|Italy                                        |3

In [6]:
"""
Base incidence rate -- weighted by link incidence
"""

query = f"""
SELECT
  COALESCE(country, "") AS country,
  COUNT(1) AS num_links
FROM isaacj.outlinks_allwikis l
LEFT JOIN {tablename} c
  ON (l.qid_to = c.qid)
WHERE
  l.snapshot = '2023-01'
  AND c.snapshot = '{wikidata_snapshot}'
GROUP BY
  COALESCE(country, "")
ORDER BY
  num_links DESC
"""

print(query)
spark.sql(query).show(300, False)


SELECT
  COALESCE(country, "") AS country,
  COUNT(1) AS num_links
FROM isaacj.outlinks_allwikis l
LEFT JOIN isaacj.qid_to_country c
  ON (l.qid_to = c.qid)
WHERE
  l.snapshot = '2023-01'
  AND c.snapshot = '2024-04-01'
GROUP BY
  COALESCE(country, "")
ORDER BY
  num_links DESC



24/04/30 17:20:18 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


+---------------------------------------------+---------+
|country                                      |num_links|
+---------------------------------------------+---------+
|France                                       |653508011|
|United States                                |467035978|
|Italy                                        |330842753|
|Mexico                                       |197337734|
|Russia                                       |180398671|
|United Kingdom                               |140974152|
|Germany                                      |123088745|
|Spain                                        |114690737|
|Japan                                        |99493950 |
|Iran                                         |83150801 |
|India                                        |73685770 |
|Ukraine                                      |61740219 |
|China                                        |60443739 |
|Poland                                       |59566468 |
|Brazil       

In [7]:
"""
Base incidence rate -- weighted by link incidence
This is the denominator for the numerators in the previous cell
"""

query = f"""
SELECT
  COUNT(1) AS num_links
FROM isaacj.outlinks_allwikis
WHERE
  snapshot = '2023-01'
"""

print(query)
spark.sql(query).show(300, False)


SELECT
  COUNT(1) AS num_links
FROM isaacj.outlinks_allwikis
WHERE
  snapshot = '2023-01'



+----------+
|num_links |
+----------+
|3522932131|
+----------+



In [9]:
"""
Base incidence rate -- weighted by link incidence
This is how many non-geographic links there are.
"""

query = f"""
SELECT
  COUNT(1) AS num_links
FROM isaacj.outlinks_allwikis l
LEFT ANTI JOIN {tablename} c
  ON (l.qid_to = c.qid AND c.snapshot = '{wikidata_snapshot}')
WHERE
  l.snapshot = '2023-01'
"""

print(query)
spark.sql(query).show(300, False)


SELECT
  COUNT(1) AS num_links
FROM isaacj.outlinks_allwikis l
LEFT ANTI JOIN isaacj.qid_to_country c
  ON (l.qid_to = c.qid AND c.snapshot = '2024-04-01')
WHERE
  l.snapshot = '2023-01'



+----------+
|num_links |
+----------+
|1320188104|
+----------+

